## Initialization

In [1]:
from datetime import datetime, timedelta
import numpy as np
import os
import pandas as pd
import requests
import sys

In [208]:
secret_token = YOUR_TOKEN

In [180]:
class KenshoGraphClient(object):
    """Basic implementation of the Kensho Graph API client"""

    def __init__(self, host, api_key, converter_func, retries=1):
        """Initialize the client"""
        self._host = host
        self._api_key = api_key
        self._retries = retries
        self._converter_func = converter_func

    def list_entity_classes(self):
        """List all available entity classes"""
        return self._get_json_or_raise('list_entity_classes')

    def list_timeline_types(self):
        """List all available entity classes"""
        return self._get_json_or_raise('list_timeline_types')

    def get_class_relationships(self, class_name):
        """Return all potential relationships of an entity of this class"""
        return self._get_json_or_raise(
            'get_class_relationships', class_name=class_name)

    def search_entities(self, class_name, search_string):
        """Search entities of the class_name that match search_string"""
        return self._get_json_or_raise(
            'search_entities', class_name=class_name, search_string=search_string)

    def list_entities_of_class(self, class_name):
        """List all entities of class"""
        return self._get_json_or_raise(
            'list_entities_of_class', class_name=class_name)

    def get_entity(self, entity_id):
        """Get entity by id"""
        return self._get_json_or_raise('get_entity', entity_id=entity_id)

    def get_related_entities(self, entity_id, relationship):
        """Get all entities related by one identified by entity_id by 'relationship'"""
        return self._get_json_or_raise(
            'get_related_entities', entity_id=entity_id, relationship=relationship)

    def get_timeline(self, timeline_id, start_date=None, end_date=None):
        """Get all events in a timeline. Optionally bound by start and end dates"""
        return self._get_json_or_raise(
            'get_timeline', timeline_id=timeline_id, start_date=start_date, end_date=end_date)

    def get_calendar(self, start_date, end_date):
        """Get all events happening in the interval [start_date, end_date)"""
        return self._get_json_or_raise('get_calendar', start_date=start_date, end_date=end_date)

    def get_ongoing_episodes(self, start_date, end_date):
        """Get all episodes ongoing in the interval [start_date, end_date)"""
        return self._get_json_or_raise(
            'get_ongoing_episodes', start_date=start_date, end_date=end_date)

    def translate_asset_id(self, id_type, asset_id):
        """Given an identification string for an asset return all known identifiers"""
        return self._get_json_or_raise('translate_asset_id', id_type=id_type, asset_id=asset_id)

    def _get_json_or_raise(self, function, **kwargs):
        """Get json from a given url. Uses get_with_retries underneath"""
        session = requests.Session()
        session.mount('https://', requests.adapters.HTTPAdapter(max_retries=self._retries))
        full_url = os.path.join(self._host, function)
        headers = {
            'Authorization': 'Token {}'.format(self._api_key),
            'Content-Type': 'application/json'
        }
        response = session.get(full_url, headers=headers, params=kwargs)
        response.raise_for_status()
        result_json = response.json()
        return self._converter_func(result_json)


def api_result_to_frame(api_result):
    """Convert API result into dataframe"""
    ordered_columns = [c['key'] for c in api_result['metadata']]
    # Create the dataframe
    df = pd.DataFrame(api_result['data'])
    existence_filtered_columns = [c for c in ordered_columns if c in df.columns]
    reordered_frame = df[existence_filtered_columns]

    for column in api_result['metadata']:
        if column['key'] not in reordered_frame.columns:
            continue
        if column['unit'] == 'DateTime' or column['unit'] == 'Date':
            reordered_frame[column['key']] = pd.to_datetime(reordered_frame[column['key']])
    return reordered_frame
    
def get_pandas_graph_client(host, api_key, retries=1):
    """Get a client that returns raw json"""
    return KenshoGraphClient(host, api_key, api_result_to_frame, retries=retries)

In [181]:
graph_client = get_pandas_graph_client('https://www.kensho.com/external/v1/', secret_token)

In [182]:
def ignore_timeline_types(df, timeline_types_to_ignore):
    if isinstance(timeline_types_to_ignore, str):
        raise ValueError('timeline_types_to_ignore may not be a string: {}'.format(timeline_types_to_ignore))
        
    indices = None
    for timeline_type in timeline_types_to_ignore:
        new_indices = df['timeline_type'] != timeline_type
        if indices is None:
            indices = new_indices
        else:
            indices &= new_indices
        
    return df[indices]

### List Functions

#### What kind of entities can I pull fromthe Kensho Knowledge Graph? 

In [183]:
graph_client.list_entity_classes()

,class_name,can_list_entities
0,Sector,True
1,Concept,True
2,TimelineType,True
3,Timeline,False
4,Region,True
5,Currency,True
6,BankMeeting,False
7,Equity,False
8,CentralBank,True


#### Nice, so you have nine broad entity classes. How do I see all of the entities that comprise a single entity class? 

##### Example One: Currencies

In [184]:
graph_client.list_entities_of_class('Currency').sort_values('entity_name').head()

,entity_name,entity_type,entity_id,entity_aliases
147,ADB Unit of Account,Currency,508bb05c-9ad2-4452-9b7e-0de2763b50dc,[XUA]
139,Afghani,Currency,78af0655-784c-4201-9fbd-ec26bcf3d0bb,[AFN]
176,Algerian Dinar,Currency,592f9df8-b0ce-482d-a48f-218f0c67f541,[DZD]
107,Argentine Peso,Currency,cfbcfa5a-ee3f-4a09-b9da-8dd3efffe186,[ARS]
1,Armenian Dram,Currency,d18b908a-130c-470e-ae94-f650ffe8f2cb,[AMD]


##### Example Two: Regions - How many countries / regions / economic regions are in the Kensho Knowledge Graph?

In [185]:
graph_client.list_entities_of_class('Region')['region_type'].value_counts()

Country           249
Region             65
EconomicRegion     19
Name: region_type, dtype: int64

#### What kind of timelines are there in the Kensho Knowledge Graph?

In [186]:
graph_client.list_timeline_types().head()

,timeline_type
0,Academia
1,Activism
2,Administrative
3,Advisory
4,All Time High


### Search Functions

#### Timelines

##### Example One: What Federal Open Markets Committee (FOMC) timelines are in the Kensho Knowledge Graph? 

In [187]:
# Plain vanilla example

graph_client.search_entities('Timeline', 'fomc')

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,Federal Open Markets Committee Meeting Minutes,Timeline,5d28d014-370c-4415-b6d3-3f1f54c0a363,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
1,Federal Open Markets Committee Meeting Statements,Timeline,bd2585f6-f5b9-4e0f-81a3-e1a77c7df4a2,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
2,Federal Open Markets Committee Meeting End,Timeline,0b2fa0d6-02e1-43db-a5c1-bfba327ad017,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
3,Federal Open Markets Committee Meeting Start,Timeline,eb5e5f23-3080-49a1-b5d8-df6f44c70bcd,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
4,"""No Hike"" FOMC Meetings",Timeline,ecccc837-43d3-4a09-9ae8-32726c1fd20f,[],Central Bank Meetings,[{'entity_id': '42aafef6-053e-4f03-af83-9c9320...
5,Federal Open Markets Committee Chair Speeches,Timeline,6a6e8724-97b0-4fc5-98c5-331727dead9e,[],Communications,[{'entity_id': '650505b8-ead1-45ee-93ba-7369a4...
6,Federal Open Markets Committee Leaves Interest...,Timeline,19bb1e6a-130a-4559-839d-ee3747f44238,[],Rate Actions,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
7,Hawkish FOMC Meetings since 1990,Timeline,989127fc-4d5d-4b21-bbaa-5bd0d2cbae3e,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
8,Dovish FOMC Meetings since 1990,Timeline,ec15d342-625e-4007-bac9-c192af95fcb5,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
9,U.S. Federal Reserve Blackout Period Start Dates,Timeline,d0261181-1287-41fe-9fff-a55626ebdea8,[],Central Bank Speeches,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...


##### Example Two: Jaysum Tatum was breaking ankles last night. Anything on the NBA? 

In [188]:
# Missile tests, food-borne illnesses and natural disasters (e.g., earthquakes and hurricanes) are other fun examples)   

graph_client.search_entities('Timeline', 'nba').head()

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,NBA Playoffs Begin,Timeline,5a676e90-f3d2-4efa-ae41-1920c51fe819,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
1,NBA Finals Start Dates,Timeline,c48f4ffb-419b-4705-ada3-8071511ad687,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
2,NBA Finals End Dates,Timeline,c87f4de0-ffd4-4663-b905-2e935cf02a7c,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
3,NBA 2K Video Game Releases,Timeline,6325a2bf-8b20-4cce-a806-a73cff01f3be,[],Product Releases,[{'entity_id': '47cb01e2-47d7-4b15-91e2-497ee8...
4,NBA All-Star Game,Timeline,01f67971-0fe5-450e-9215-dac10e2b1366,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...


##### Stress test the Graph - i.e., search for something you don't think we have. Let me know if you're unable to find something of interest.  

### Get Functions

##### The Graph had ~3MM entities and ~4.6MM relationship as of April 2018. What kind of relationships are there for various equities? 

In [189]:
graph_client.get_class_relationships('Equity')

,relationship_id,related_class
0,EquityConcepts,Concept
1,Peers,Equity
2,EquitySuppliers,Equity
3,EquityTimelines,Timeline
4,EquityConsumers,Equity


##### Let's take a look at Apple:

In [190]:
graph_client.search_entities('Equity', 'AAPL')

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Apple Inc. (AAPL),Equity,f9d09c10-c6de-4e8f-9624-996774888bcc,"[Apple Computers, Apple comp, Apple Computer, ...",AAPL,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3


In [191]:
aapl_id = graph_client.search_entities('Equity', 'AAPL')['entity_id'][0]

##### Example One: Concepts - What industries is Apple involved in? 

In [192]:
aapl_concepts = graph_client.get_related_entities(aapl_id, 'EquityConcepts')
aapl_concepts

,entity_name,entity_type,entity_id,entity_aliases
0,Smartphones,Concept,fa23a8bc-07e2-4bf2-970c-de68b7c3f431,"[smartphone, smart phone, smart phones]"
1,Tablets,Concept,8a384f5c-4673-4c5e-a776-4780f4316161,[tablet computers]
2,Wearables,Concept,1f0667ed-3009-463a-9301-bfe158f863a8,"[fashion technology, fashion electronics, wear..."
3,Digital Assistants,Concept,c8114c49-9a5a-4ab1-b152-f03e99aa5731,[virtual assistant]
4,Smart Homes,Concept,5ef8c951-f859-463a-9b69-e0cd55d1dace,"[home automation, connected home, internet of ..."
5,Digital Payments,Concept,96d1bc76-938e-41a2-ac17-be05a71c98ae,"[mobile payments, e-commerce payment system, o..."
6,4K TV,Concept,83cad2d0-fed4-4d58-8922-5302dd74307a,"[4k resolution, 4k ultra high definition, 4k uhd]"
7,Streaming Music,Concept,505b8e2a-bef4-4452-83c6-8635acb8462a,"[streaming music services, music streaming ser..."
8,Data Storage,Concept,b17035bf-de7c-4b6b-941f-1badb581b985,"[data management, cloud storage]"
9,Virtual Reality,Concept,2eaea590-2764-49ea-b64c-39c340e3b4f9,[vr]


##### Example Two: Peers - Who are Apple's competitors? 

In [193]:
aapl_peers = graph_client.get_related_entities(aapl_id, 'Peers')
aapl_peers

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,"Amazon.com, Inc. (AMZN)",Equity,cfb73e32-4fa8-4701-898c-6c39279bbdb4,"[Amazon.com, Inc., Amazon company, Amazon Web ...",AMZN,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089
1,"Facebook, Inc. (FB)",Equity,d8ce5287-c2c4-4c5a-8ec7-334b8213de4b,"[Thefacebook.com, The Facebook, Facebook, Inc....",FB,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
2,Microsoft Corporation (MSFT),Equity,06826ee5-f5e2-4342-9bee-1a546db3b03b,"[Mikrosoft, Windows Corporation, Microsoft.com...",MSFT,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
3,"Netflix, Inc. (NFLX)",Equity,8145cd22-43cb-4911-8b91-d3fa8f69cc9d,"[Netflix Inc., Netflix.com, NetFlicks, Net Fli...",NFLX,Movies & Entertainment,9e1fd6e9-4d40-4e8a-b60a-30e971d72b63
4,Altaba Inc. (AABA),Equity,6b7f93a8-cb9b-4358-a2a8-a1152377f4fc,"[Yahoo! Inc., Yahoo Alibaba Group, Yahoo, Yahoo!]",AABA,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089


##### Example Three: Suppliers - Tell me about Apple's supply chain.  

In [194]:
aapl_suppliers = graph_client.get_related_entities(aapl_id, 'EquitySuppliers')
aapl_suppliers.sort_values('entity_name').head()

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
51,AAC Technologies Holdings Inc. (2018),Equity,2347335c-477e-40ab-b4e5-6a51e2384f7f,[],2018,Electronic Manufacturing Services,9cba514c-b902-4623-9e20-f8402dad25f1
36,APG|SGA SA (APGN),Equity,9062ee9a-984b-4273-a4f6-46f1a08f73bc,[],APGN,Advertising,f83e6b06-b017-4e63-b9dc-9cd5eebf37b2
89,"ASE Industrial Holding Co., Ltd. (ASX)",Equity,5918372a-3a1f-49cb-afb1-206d5c0bbbad,[],ASX,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
24,AU Optronics Corp. (2409),Equity,7af015af-5334-4040-8fce-dd77773bfaec,[],2409,Electronic Components,19a722a6-e905-4656-945c-1d5022b06757
23,AU Optronics Corp. (AUO),Equity,683203d0-67cb-4258-897d-2cc397f0dcba,[],AUO,Electronic Components,19a722a6-e905-4656-945c-1d5022b06757


##### Apple suppliers are (unsurprisingly) predominately semiconductor and electronic component companies. 

In [195]:
aapl_suppliers['sector_name'].value_counts()

Semiconductors                                27
Electronic Components                         18
Electrical Components & Equipment              7
Consumer Electronics                           6
Electronic Manufacturing Services              5
Technology Hardware, Storage & Peripherals     5
Electronic Equipment & Instruments             4
Automobile Manufacturers                       2
Building Products                              2
Auto Parts & Equipment                         2
Advertising                                    2
Application Software                           2
Industrial Machinery                           1
Internet Services & Infrastructure             1
Diversified Chemicals                          1
Industrial Conglomerates                       1
Diversified Support Services                   1
Interactive Home Entertainment                 1
Aerospace & Defense                            1
Diversified Real Estate Activities             1
Semiconductor Equipm

##### Example Four: Consumers - Who are the major corporate consumers of Apple products?  

In [196]:
aapl_consumers = graph_client.get_related_entities(aapl_id, 'EquityConsumers')
aapl_consumers.sort_values('entity_name').head()

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
1,United States Cellular Corporation (USM),Equity,c1faf554-401c-4ec1-9482-306143bb5581,[],USM,Wireless Telecommunication Services,558a4212-6576-4fe6-b121-21e0eb91edb9
0,Verizon Communications Inc. (VZ),Equity,0f154c5b-90a2-40fc-89a0-c9fce7d05676,"[Verizon, Verizon VoiceWing, VZN, Verizon.net,...",VZ,Integrated Telecommunication Services,3153749b-509c-48ba-9705-eafa1d23d71a


##### Example Five: Events & Timelines - What Apple events are there in the Kensho Knowledge Graph? 

In [197]:
# Undersold breadth of AAPL event sets - we have 108 distinct event sets

graph_client.get_related_entities('f9d09c10-c6de-4e8f-9624-996774888bcc', 'EquityTimelines').head()

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,Apple Inc. (AAPL) Earnings Releases,Timeline,ec5a179d-ae08-4912-8bca-6a348baa2213,[],Earnings,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
1,Apple Inc. moves by 5% or more in a day,Timeline,c5234740-2695-4c31-a9fd-e8c1c6559256,[],Big Moves,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
2,Apple Inc. Corporate Guidance - Lowered,Timeline,7244135a-f39a-4443-94ed-5508b580cf9d,[],Corporate Guidance - Lowered,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
3,Apple Inc. Corporate Guidance - Raised,Timeline,e8479f2e-4cc4-43f7-8d9a-b3e8d056b4a0,[],Corporate Guidance - Raised,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
4,Apple Inc. Corporate Guidance - New/Confirmed,Timeline,96b9e238-45f1-4949-855e-04312153f551,[],Corporate Guidance - New/Confirmed,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...


### More Events & Timelines 

#### What events (past or upcoming) should I know about? 

In [198]:
# We limit get_calendar pulls to ten days for throttling reasons - this is how I sidestep it: 

start_dates = []
end_dates = []

start = datetime.today() - timedelta(63) # Using 63 days as a proxy for a quarter - pass in whatever lookback period you need  

while start < datetime.today(): # While loop to prevent the function from pulling all future events already loaded into the Graph 
    start_dates.append(start.strftime('%Y-%m-%d'))
    
    end = start + timedelta(10)
    end_dates.append(end.strftime('%Y-%m-%d'))
    
    start = end + timedelta(1)

In [199]:
trailing_quarter_events = pd.DataFrame()

for start, end in zip(start_dates, end_dates):
    temp_frame = graph_client.get_calendar(start, end)
    trailing_quarter_events = pd.concat((trailing_quarter_events, temp_frame), axis=0)

In [200]:
trailing_quarter_events.shape

(45116, 8)

#### 45,000+ events from the past quarter alone.. now let's get more surgical:

##### Narrowing by TimelineType

In [201]:
ttypes = graph_client.list_entities_of_class('TimelineType')

# Instantiating a dict to streamline id searching and saving

ttypes_dict = dict(zip(ttypes['entity_name'], ttypes['entity_id'])) # ttypes_dict includes all 212 TimelineTypes in the Graph 

# For example: 

political_speeches_id = ttypes_dict['Political Speeches']
dividend_decreases_id = ttypes_dict['Dividend Decreases']

In [202]:
# You can also pass them in as strings, which we'll do here:

interesting_event_types = ['Political Speeches', 'Dividend Decreases', 'Big Moves']

trailing_quarter_events[trailing_quarter_events['timeline_type'].isin(interesting_event_types)]['timeline_type'].value_counts()

Dividend Decreases    353
Big Moves             330
Political Speeches    226
Name: timeline_type, dtype: int64

##### Narrowing by geography.. 

In [203]:
# Saving down entity_ids for regions of interest (e.g., Japan and Canada)

japan_id = graph_client.search_entities('Region', 'Japan')['entity_id'][0]
canada_id = graph_client.search_entities('Region', 'Canada')['entity_id'][0]

In [204]:
japan_bool = []
canada_bool = []

for entities_list in trailing_quarter_events['related_entities']: 
    related_to_japan = 0
    related_to_canada = 0
    
    for entity in entities_list:
        if entity['entity_id'] == japan_id:
            related_to_japan = 1
        elif entity['entity_id'] == canada_id:
            related_to_canada = 1
            
    japan_bool.append(related_to_japan)
    canada_bool.append(related_to_canada)

trailing_quarter_events['japan_bool'] = japan_bool
trailing_quarter_events['canada_bool'] = canada_bool

In [205]:
# Returns all events from last week related to Japan and Canada 

trailing_quarter_events[(trailing_quarter_events['japan_bool']==1) | (trailing_quarter_events['canada_bool']==1)].head()

,event_date,event_date_precision,event_name,event_description,timeline_id,timeline_name,timeline_type,related_entities,japan_bool,canada_bool
84,2018-08-22 00:00:00,YYYY-MM-DD,June 2018 Monthly Labour Survey Released,,67c23ff5-4995-443b-89fd-3f5de3c456b0,Japan Monthly Labour Survey Releases,Labor,[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,1,0
266,2018-08-22 04:30:00,YYYY-MM-DDTHH:mmZ,All Industry Index: Month over Month (Japan) R...,,e1ee722a-983b-4179-9035-1bf846080c1b,All Industry Index: Month over Month (Japan),Business Activity,[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,1,0
271,2018-08-22 04:30:00,YYYY-MM-DDTHH:mmZ,All Industry Index: Year over Year (Japan) Rel...,,b3be9ab8-535c-4b42-947d-f64294d02d82,All Industry Index: Year over Year (Japan),Business Activity,[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,1,0
449,2018-08-22 09:23:39,YYYY-MM-DDTHH:mm:ssZ,"M 4.5 - 11km NNW of Uto, Japan",,ce785cac-226b-44c4-a944-97443bc073b5,Japan Earthquakes,Natural Disasters,[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,1,0
547,2018-08-22 12:30:00,YYYY-MM-DDTHH:mmZ,Retail Sales: Year over Year (Canada) Release ...,,43d5c0c6-cbde-4f96-aac7-84d9db411412,Retail Sales: Year over Year (Canada),Retail,[{'entity_id': '8545e2a7-c1f3-4e2e-8984-96541e...,0,1


##### Narrowing by ticker list (e.g., XLK constituents) 

In [258]:
# CSV attached in email - workaround until we integrate constituent data from S&P

xlk_constituents = pd.read_csv('spdr_constituents_10.22.2018.csv')['XLK'].dropna().tolist()

In [244]:
xlk_constituent_uuids = []

for constituent in xlk_constituents: 
    try:
        xlk_constituent_uuid = graph_client.search_entities('Equity', constituent)['entity_id'].values[0]
        xlk_constituent_uuids.append(xlk_constituent_uuid)   
    except:
        False

In [245]:
xlk_constituent_uuids_set = set(xlk_constituent_uuids)

In [251]:
def xlk_uuid_in_row(row):
    '''
       Uses intersection to determine if any of the 61 XLK constituents as of 10.22.2018 are referenced in a row - 0 if no, 1 if yes 
    '''
    return len(set([entity['entity_id'] for entity in row]) & xlk_constituent_uuids_set) > 0

In [252]:
trailing_quarter_events['xlk_bool'] = trailing_quarter_events['related_entities'].apply(graph_uuid_in_row).astype(int)

In [253]:
# 753 events tagged to XLK constituents over the past (trailing quarter)

trailing_quarter_events['xlk_bool'].value_counts()

0    44363
1      753
Name: xlk_bool, dtype: int64

In [255]:
xlk_mask = trailing_quarter_events['related_entities'].apply(graph_uuid_in_row).astype(bool)

In [257]:
trailing_quarter_events.loc[xlk_mask].head()

,event_date,event_date_precision,event_name,event_description,timeline_id,timeline_name,timeline_type,related_entities,japan_bool,canada_bool,xlk_bool
1,2018-08-22,YYYY-MM-DD,Reuters reports Apple to gain unconditional EU...,Reuters reports that Apple will gain unconditi...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...,0,0,1
5,2018-08-22,YYYY-MM-DD,Nvidia claims RTX 2080 is up to 50 percent fas...,Nvidia reveals that there will be performance ...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '56da58ef-10f5-4333-a7b0-f06c3f...,0,0,1
9,2018-08-22,YYYY-MM-DD,The Wall Street Journal reports Facebook volun...,The Wall Street Journal reports that Facebook ...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '5fa35db7-3437-4853-9802-4a6490...,0,0,1
11,2018-08-22,YYYY-MM-DD,"Apple CEO Tim Cook to receive 560,000 company ...",Apple Chief Executive Officer Tim Cook will re...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...,0,0,1
12,2018-08-22,YYYY-MM-DD,Windows Central reports Microsoft may be plann...,"According to a report by Windows Central, Micr...",cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '56da58ef-10f5-4333-a7b0-f06c3f...,0,0,1


#### Apple Buybacks

In [238]:
aapl_event_sets = graph_client.get_related_entities(aapl_id, 'EquityTimelines')

In [116]:
aapl_buyback_mask = aapl_event_sets['entity_name'].str.contains('uyback')

In [117]:
aapl_event_sets.loc[aapl_buyback_mask]

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
13,Apple Inc. Buyback - Change in Plan Terms,Timeline,db0c4d86-92ba-4bd2-8420-fcffa57ec503,[],Buyback - Change in Plan Terms,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
14,Apple Inc. Buyback Tranche Update,Timeline,eb12cd47-1425-4358-8005-65e269d9c381,[],Buyback Tranche Update,[{'entity_id': '25014370-ce33-4e5c-8bde-14facb...
17,Apple Inc. Buyback Transaction Announcements,Timeline,52813076-9932-48bb-8bf4-559558a8407f,[],Buyback Transaction Announcements,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
37,Apple Inc. Buyback Announcements,Timeline,afe3fe83-110b-42a8-b9be-4152099e8308,[],Buyback Announcements by Company,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
38,Apple Inc. Buyback Closings,Timeline,ef328fdd-5388-4fc4-bfb2-46674142adf1,[],Buyback Closings by Company,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...


In [118]:
# Making another mask for the specific buyback timeline I'm interested in: 

aapl_buyback_announcement_mask = aapl_event_sets['entity_name'].str.contains('Buyback Transaction Announcement')

In [119]:
aapl_buyback_announcement_id = aapl_event_sets.loc[aapl_buyback_announcement_mask, 'entity_id'].values[0]

In [120]:
# Now pulling down this specific timeline

graph_client.get_timeline(aapl_buyback_announcement_id)

C:\Users\keenanfreyberg\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,event_date,event_date_precision,event_name,event_description,primary_provenance,provenance,related_entities
0,2012-03-19 12:36:00,YYYY-MM-DDTHH:mmZ,Apple Inc. (NasdaqGS:AAPL) announces an Equity...,The Board of Directors of Apple Inc. (NasdaqGS...,[],[],[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
1,2018-05-01 00:00:00,YYYY-MM-DD,Apple Inc. authorizes a Buyback Plan.,The Board of Directors of Apple Inc. has autho...,[],[],[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
2,2018-05-01 20:30:00,YYYY-MM-DDTHH:mmZ,Apple Inc. (NasdaqGS:AAPL) announces an Equity...,Apple Inc. (NasdaqGS:AAPL) announces a share r...,[],[],[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
